In [1]:
from pyspark import SQLContext, SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, row_number
import pyspark.sql.functions as f
from pyspark.sql.window import Windowes import LongType
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
import math

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
367,application_1654871637433_4345,pyspark,idle,Link,Link,ds-gliu,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# april_2022 = "s3://ojo-data-science/movoto-data/big_query/webanalytics/sessions_hits_newschema_v5/2022/04/*/"
may_2022 = "s3://ojo-data-science/movoto-data/big_query/webanalytics/sessions_hits_newschema_v5/2022/05/*/"
june_2022 = "s3://ojo-data-science/movoto-data/big_query/webanalytics/sessions_hits_newschema_v5/2022/06/*/"
infiles = [may_2022, june_2022]
df = spark.read.parquet(*infiles) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Filter out non-listing views activity
df = df.filter(col("hits_page_pageTitle") == "Property Detail")
df = df.filter(col("hits_customDimensions_index_64") != "")
df = df.filter(col("hits_isInteraction") == "TRUE")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Example URL: "https://www.movoto.com/home/5021-20th-street-ct-e-bradenton-fl-34203-431_a4539073"
# MLS ID: 431
# Property ID: a4539073
# Property zipcode: 34203
# Property address: 5021-20th-street-ct-e-bradenton-fl-34203-431_a4539073

# Get MLS ID
split_col = f.split(df['hits_customDimensions_index_64'], '_')
df = df.withColumn('mls_id', f.substring(split_col.getItem(0), -3, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Get property id
split_col = f.split(df['hits_customdimensions_index_64'], '_')
df = df.withColumn('mls_property_id', f.substring(split_col.getItem(1), 1, 10))
    
# Clean up
df = df.withColumn("mls_property_id", f.when(df.mls_property_id.contains('/'), 
                                        f.expr("substring(mls_property_id, 1, length(mls_property_id)-3)"))\
                                        .otherwise(df.mls_property_id))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Get property zipcode
split_col = f.split(df['hits_customdimensions_index_64'], '_')
df = df.withColumn("cut", f.substring(split_col.getItem(0), -9, 9))
split_again = f.split(df["cut"], "-|/")
df = df.withColumn("col_len", f.size(split_again))
df = df.withColumn("property_zipcode", f.when(df.col_len == 2, split_again.getItem(0)).otherwise(split_again.getItem(1)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Get property address
split_col = f.split(df['hits_customdimensions_index_64'], '/')
df = df.withColumn('property_address', split_col.getItem(4))
# Clean up address
split_col = f.split(df['hits_customdimensions_index_64'], '_')
df = df.withColumn("property_address", f.when(df.property_address.contains('_'), 
                                        f.expr("substring(property_address, 1, length(property_address)-12)"))\
                                        .otherwise(df.property_address))

df = df.withColumn("property_address", f.when(df['property_address'].substr(-3, 1) == '-', 
                                       f.expr("substring(property_address, 1, length(property_address)-3)"))\
                                        .otherwise(df.property_address))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Remove nulls
df = df.withColumn("zip-value", f.col("property_zipcode").cast("float").isNotNull())\
    .filter(col('zip-value') == 'true')\
    .drop("zip-value")

# Remove zips with length != 5
df = df.withColumn("text_left_over", f.length("property_zipcode"))\
    .filter(col('text_left_over') == 5)\
    .drop("text_left_over")

# Remove successive duplicate zipcodes
df = df.withColumn("uniqueVisitId", f.concat(f.col("visitid"), f.col("fullvisitorId")))
window = Window.partitionBy("uniqueVisitId").orderBy("hits_hitnumber")

# If the last zipcode is equal to the current, give False; otherwise True
get_last = f.when(f.lag("property_zipcode", 1).over(window) == col("property_zipcode"), False).otherwise(True)
df = df.withColumn("property_zipcode_drop_dups", get_last)\
        .filter(col("property_zipcode_drop_dups"))\
        .drop("property_zipcode_drop_dups")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Rationale: users must spend at least 45 seconds on a listing in order for it to be a quality click

# Get hits_time for next listing 
df = df.withColumn("lead_hits_time", f.lead("hits_time", 1).over(window))

# Create new column of differences and filter 
df = df.withColumn("time_spent", f.col("lead_hits_time") - f.col("hits_time"))\
        .filter(col("time_spent") >= 45)\
        .drop("lead_hits_time", "time_spent")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Develop sentences
sentences = df.groupBy('uniqueVisitId').agg(f.collect_list("property_zipcode")) 

list_col = "collect_list(property_zipcode)"

# Get first character of every sentence **as array
def get_first_char(my_list):
    return str(my_list[0])[0]

get_first_char_udf = udf(get_first_char, StringType())

sentences = sentences.withColumn("first_char", get_first_char_udf(sentences[list_col]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sentences.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+----------+-------------------------------------------------------------------------+
|uniqueVisitId                |first_char|sentences_string                                                         |
+-----------------------------+----------+-------------------------------------------------------------------------+
|1648793792147401526900857024 |4         |[43207,43205]                                                            |
|16487953062903547302760230786|9         |[95210,95203]                                                            |
|16487965022357016614175307802|1         |[15045]                                                                  |
|1648796574826153612616103662 |2         |[28560]                                                                  |
|16487965915711311142362664613|2         |[22152,22003]                                                            |
|16487967507527788867815383376|3         |[33321]               

In [12]:
# Convert sentences_array to a string
def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string, StringType())

sentences = sentences.withColumn("sentences_string", array_to_string_udf(sentences['collect_list(property_zipcode)']))\
                        .drop("collect_list(property_zipcode)")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Read to csv file
PATH = "s3://ojo-data-science/glen/clickstream_v2_clean"
sentences.write\
            .mode("overwrite")\
            .csv(PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…